# Test Vidéo

L'objectif est de maintenant tester notre modèle en temps réel. Nous reprenons la même structure que pour l'acquisition des données pour l'entrainement du modèle. A la différence que nous ajoutons la prédiction du signe effectué avec la main et affichons la lettre pour former un mot.

Afin d'avoir un résultat satisfaisant, le test doit être réalisé sur un fond uniforme pour éviter les artefacts parasites. Bien qu'à l'ouverture de la caméra, la main n'est pas encore placé dans le cadre de détection, le modèle prédit de façon chaotique des lettres du fait d'une trop grande sensibilité.

Certaines lettres sont plus difficilement déchiffrables que d'autres (notamment le F, K, M, N, T), nous avons ajouté une commande afin d'effacer la dernière lettre si elle n'était pas recherchée. Nous avons pu construire certains mots, et phrases (Pour des raisons de visibilité, nous avons changé l'espace par un underscore).

In [1]:
import numpy as np
import cv2
import tensorflow as tf

### Chargement du modèle

In [2]:
model = tf.keras.models.load_model(r"model5_sign")

In [3]:
background = None
accumulated_weight = 0.5
ROI_top = 50
ROI_bottom = 300
ROI_right = 50
ROI_left = 300

In [25]:
word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z',26:' _ '}

In [5]:
def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [6]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    thresholded = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    
    return (thresholded)

In [52]:
# Ouverture de la caméra
cam = cv2.VideoCapture(0)

# Définition des variables (nombre de frame, lettres prédites, mot à afficher, liste pour l'ajout de la lettre prédite)
num_frames = 0
letter = ''
word = ''
append_text = []


while True:
    ret, frame = cam.read()
    
    # Eviter l'inversement de l'image 
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    
    # Traitement
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (7, 7), 0)
    
    # Détection du Background
    if num_frames < 70:
        cal_accum_avg(gray_frame, accumulated_weight)
        cv2.putText(frame_copy, "Chargement", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # Voir si la main est bien détectée
        hand = segment_hand(gray_frame)
        
        if hand is not None:
            
            thresholded = hand
            
            cv2.imshow("Filtre", thresholded)
            
            # Traitement
            thresholded = cv2.resize(thresholded, (128, 128))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            # Incorporation du modèle (prédiction et les probabilités de celle-ci)
            pred = model.predict(thresholded)
            proba = model.predict_proba(thresholded)
            
            # Affichage de la prédiction
            cv2.putText(frame_copy, word_dict[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            # Ajout de la lettre au mot qu'on veut former (lettre avec une probabilité de 70% minimum)
            if np.argmax(proba)>0.7:
                letter = word_dict[np.argmax(pred)]
                append_text.append(letter)
                
                # Si cette lettre est présente plus de 15 fois dans la liste append_text, alors elle est ajouté au mot formé
                if append_text.count(letter)>35:
                    word+=letter
                    letter=''
                    append_text = []
                    
            # Efface la dernière lettre du mot
            if cv2.waitKey(1) & 0xFF == ord('e'):
                word = word[0:-1]
                    
            cv2.putText(frame_copy, word, (250, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                                                    
    # Dessine une ROI de la frame Filtre
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)
    num_frames += 1
    
    # Affiche la frame qui permettra la prédiction
    cv2.putText(frame_copy, "Prediction", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Camera", frame_copy)
    
    # Fermer l'application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()